In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time

import torch.nn.functional as F

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
import numpy as np
import random
import os


def set_seed(seed=42):
    """
    Set random seeds for reproducibility across multiple libraries.

    Args
    ----
    seed (int):
        Seed value to use
    """
    # Set PyTorch seed
    torch.manual_seed(seed)

    # Set CUDA seed (if available)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # for multi-GPU setups

    # Set NumPy seed
    np.random.seed(seed)

    # Set Python's random seed
    random.seed(seed)

    # Set environment variable for some PyTorch operations
    os.environ["PYTHONHASHSEED"] = str(seed)


# Example usage
set_seed(42)  # or any other seed value

In [4]:
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate, bn_size, drop_rate):
        super(DenseLayer, self).__init__()
        # BN-ReLU-Conv(1x1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(
            in_channels, bn_size * growth_rate, kernel_size=1, stride=1, bias=False
        )

        # BN-ReLU-Conv(3x3)
        self.bn2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            bn_size * growth_rate,
            growth_rate,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False,
        )

        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = self.conv1(self.relu1(self.bn1(x)))
        new_features = self.conv2(self.relu2(self.bn2(new_features)))
        if self.drop_rate > 0:
            new_features = F.dropout(
                new_features, p=self.drop_rate, training=self.training
            )
        return torch.cat([x, new_features], 1)


class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, bn_size, drop_rate):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.add_module(
                "denselayer%d" % (i + 1),
                DenseLayer(
                    in_channels + i * growth_rate, growth_rate, bn_size, drop_rate
                ),
            )

    def forward(self, x):
        features = x
        for layer in self.layers:
            features = layer(features)
        return features


class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(
            in_channels, out_channels, kernel_size=1, stride=1, bias=False
        )
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv(x)
        x = self.pool(x)
        return x


class DenseNet(nn.Module):
    def __init__(
        self,
        growth_rate=12,
        block_config=(6, 12, 8),
        num_init_features=32,
        bn_size=4,
        drop_rate=0.2,
        num_classes=10,
    ):
        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(
            nn.Conv2d(
                3, num_init_features, kernel_size=3, stride=1, padding=1, bias=False
            ),
            nn.BatchNorm2d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            # Add a dense block
            block = DenseBlock(
                num_layers=num_layers,
                in_channels=num_features,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
            )
            self.features.add_module("denseblock%d" % (i + 1), block)
            num_features = num_features + num_layers * growth_rate

            # Add a transition layer between dense blocks (except after the last block)
            if i != len(block_config) - 1:
                trans = Transition(
                    in_channels=num_features, out_channels=num_features // 2
                )
                self.features.add_module("transition%d" % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module("norm5", nn.BatchNorm2d(num_features))
        self.features.add_module("relu5", nn.ReLU(inplace=True))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.adaptive_avg_pool2d(features, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out




In [5]:
class EarlyStopper:
    def __init__(self, patience=2, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_error = float("inf")

    def check(self, val_error, model):
        if val_error < self.min_error - self.min_delta:
            self.min_error = val_error
            torch.save(model, "best_model.pth")
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [6]:
model = DenseNet(growth_rate=12, block_config=(6, 12, 8), num_classes=10)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.95, weight_decay=0.0005)
# Alternative: optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


import os
import json
import time

ES = EarlyStopper(patience=4, min_delta=0)


def train_and_saved_logged(
    trainloader,
    valloader,
    path,
    proportion,
    epochs=5,
):
    time_started = time.time()
    _train_errors = []
    _val_errors = []

    json_log = {
        "metadata": {
            "model": str(model),
            "freq_bins": 64,
            "time_steps": 64,
            "batch_size": trainloader.batch_size,
            "train_set_size": len(trainloader.dataset),
            "optimizer": type(optimizer).__name__,
            "loss_function": str(criterion),
            "num_epochs": epochs,
            "proportion": proportion,
        },
        "data": {},
    }

    os.makedirs(path, exist_ok=True)

    for epoch in range(epochs):
        correct_train = 0
        total = 0
        running_loss = 0.0

        epoch_loss = 0.0
        total_batches = 0

        model.train()
        print(f"Epoch {epoch + 1}/{epochs}")
        for i, data in enumerate(trainloader, 0):

            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            correct_train += (torch.argmax(outputs, 1) == labels).float().sum().item()
            total += len(outputs)

            running_loss += loss.item()
            epoch_loss += loss.item()
            total_batches += 1

            if i % 200 == 199:
                acc = correct_train / total
                print(
                    f"[{epoch + 1}, {i + 1}], time {time.time() - time_started:.2f}s loss: {running_loss / 200:.3f}, acc: {acc * 100:.2f}%"
                )
                running_loss = 0.0
        print(f"Epoch {epoch + 1} completed.")
        train_acc = correct_train / total
        _train_errors.append(epoch_loss / total_batches)
        val_error = 0
        correct_val = 0

        model.eval()
        print("start evaluating for epoch: ", epoch + 1)
        counter_image = 0
        with torch.no_grad():
            for images, labels in valloader:
                counter_image += len(images)
                if counter_image % 10000 == 0: 
                    print("evaluating images: ", counter_image)

                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_error += criterion(outputs, labels).item() * images.size(0)
                correct_val += (torch.argmax(outputs, 1) == labels).float().sum().item()

        val_error = val_error / len(valloader.dataset)
        val_acc = correct_val / len(valloader.dataset)
        _val_errors.append(val_error)

        print(
            f"epoch {epoch + 1} TRAIN error: {epoch_loss / total_batches:.4f}, acc: {train_acc:.4f}"
        )
        print(
            f"epoch {epoch + 1} VALIDATION error: {val_error:.4f}, acc: {val_acc:.4f}"
        )

        # Log epoch summary to JSON
        json_log["data"][str(epoch + 1)] = {
            "batches": {},  # Leave empty if not tracking batch-level stats
            "summary": {
                "train_loss" : epoch_loss / total_batches,
                "train_acc": train_acc,
                "val_loss": val_error,
                "val_acc": val_acc,
                "lr": optimizer.param_groups[0]["lr"],
            },
        }
        model_name = "DenseNet"
        timestamp = time.strftime("%Y%m%d-%H%M%S")

        json_filename = f"training_log_{model_name}_proportion{proportion}.json"

        with open(os.path.join(path, json_filename), "w") as f:
            json.dump(json_log, f, indent=4)

        if ES.check(val_error, model):
            print("Early stopping triggered!")
            break

In [7]:
list_size_of_train = [1, 2, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset



transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.ImageFolder(
    "../../data/raw/train/", transform=transform
)
valset = torchvision.datasets.ImageFolder("../../data/raw/valid/", transform=transform)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=32, shuffle=True, num_workers=2
)
valloader = torch.utils.data.DataLoader(
    valset, batch_size=32, shuffle=False, num_workers=2
)



for proportion in list_size_of_train :
    print("proportion", proportion, "%")


    n_total = len(trainset)
    n_subset = int(n_total * (proportion / 100.0))

    
    indices = torch.randperm(n_total)[:n_subset]

    
    train_subset = Subset(trainset, indices)

    
    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=32, shuffle=True, num_workers=2
    )

    print("trainloader", len(trainloader.dataset))

    train_and_saved_logged(
        trainloader,
        valloader,
        path="./logs/",
        proportion=proportion,
        epochs=60,
    )

    

proportion 1 %
trainloader 900
Epoch 1/60
Epoch 1 completed.
start evaluating for epoch:  1
evaluating images:  20000
evaluating images:  40000
evaluating images:  60000
evaluating images:  80000
evaluating images:  90000
epoch 1 TRAIN error: 1.7178, acc: 0.3678
epoch 1 VALIDATION error: 1.7263, acc: 0.3450
Epoch 2/60
Epoch 2 completed.
start evaluating for epoch:  2
evaluating images:  20000
evaluating images:  40000
evaluating images:  60000
evaluating images:  80000
evaluating images:  90000
epoch 2 TRAIN error: 1.7052, acc: 0.3856
epoch 2 VALIDATION error: 1.8542, acc: 0.3139
Epoch 3/60
Epoch 3 completed.
start evaluating for epoch:  3
evaluating images:  20000
evaluating images:  40000
evaluating images:  60000
evaluating images:  80000
evaluating images:  90000
epoch 3 TRAIN error: 1.6828, acc: 0.3822
epoch 3 VALIDATION error: 1.7369, acc: 0.3410
Epoch 4/60
Epoch 4 completed.
start evaluating for epoch:  4
evaluating images:  20000
evaluating images:  40000
evaluating images:  60